In [1]:

# Set up notebook
%pprint
%matplotlib inline
import sys
import os.path as osp, os as os

executable_path = sys.executable
scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts')
py_folder = osp.abspath(osp.join(os.pardir, 'py')); assert osp.exists(py_folder), "Create the py folder"
ffmpeg_folder = r'C:\ffmpeg\bin'
shared_folder = osp.abspath(osp.join(os.pardir, 'share')); assert osp.exists(shared_folder)
scripts312_folder = r'C:\Users\daveb\AppData\Roaming\Python\Python312\Scripts'

if osp.exists(scripts_folder) and (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if osp.exists(ffmpeg_folder) and (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)
if osp.exists(scripts312_folder) and (scripts312_folder not in sys.path): sys.path.insert(1, scripts312_folder)

from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)
nu.delete_ipynb_checkpoint_folders()

Pretty printing has been turned OFF


In [2]:

from tqdm import tqdm
from colormath.color_objects import LabColor
import networkx as nx
from itertools import combinations
import geopandas as gpd
import matplotlib.pyplot as plt
import webcolors
from cycler import cycler

def get_hex_color_dictionary(country_names, verbose=True):
    
    # Get the fixed point in the CIELAB color space
    ranges = [(0, 100), (-128, 127), (-128, 127)]
    ocean_color = (0.5294117647058824, 0.807843137254902, 0.9215686274509803)  # light blue
    fixed_point = nu.rgb_to_lab(ocean_color).get_value_tuple()
    
    # Get additional points to spread
    point_count = len(country_names)
    
    # Initialize a list to store trial results
    trials = []
    total_trials = 5  # Adjust this based on your patience

    # Initialize the progress bar only if verbose is True
    if verbose:
        pbar = tqdm(total=total_trials)
    else:
        pbar = None  # No progress bar
    
    # Perform trials to find the best spread of points
    while len(trials) < total_trials:
        
        # Attempt to spread the points evenly within the color space
        try:
            spread_points = nu.spread_points_in_cube(
                point_count, fixed_point, *ranges, verbose=False
            )

            # Ensure spread points have all unique XKCD names
            xkcd_set = set()
            for lab_color in spread_points:
                rgb_color = nu.lab_to_rgb(LabColor(*lab_color))
                nearest_neighbor = nu.get_nearest_neighbor(rgb_color, nu.xkcd_colors)
                xkcd_set.add(nu.nearest_xkcd_name_dict[nearest_neighbor])
            if len(xkcd_set) == len(spread_points):
                
                # Measure how far the points are from the fixed point
                spread_value = nu.calculate_spread(spread_points[1:], fixed_point, verbose=False)
                
                # Store the result as a tuple of (spread_points, spread_value)
                trial_tuple = (spread_points, spread_value)
                trials.append(trial_tuple)  # Add a new trial
                
                # Update the progress bar if it exists
                if pbar:
                    pbar.update(1)  # Increment the progress bar by 1
        
        # If an error occurs (e.g., a spread point too close to black or white), skip this trial
        except Exception as e:
            if verbose:
                raise(Exception, f'{e.__class__.__name__} error attempting to spread the points evenly within the color space: {e}')
            continue

    # Close the progress bar if it was used
    if pbar:
        pbar.close()
        
    # Select the trial with points as far away from the fixed point as possible
    trial_tuple = max(trials, key=lambda x: x[1])
    
    # Extract the spread points from the best trial
    spread_points = [nu.lab_to_rgb(LabColor(*lab_color)) for lab_color in trial_tuple[0]]
    
    # Download all the vector and raster map data from Natural Earth
    file_path = osp.abspath(osp.join(nu.data_folder, 'zip', 'ne_10m_admin_0_countries.zip'))
    countries_df = gpd.read_file(file_path)
    
    # Get the geometry of each country in a dictionary
    geometry_dict = {}
    
    # For each country, get the coordinates of the grid it intersects with
    mask_series = countries_df.NAME_EN.isin(country_names)
    for (country_name, geometry), df in countries_df[mask_series].groupby(['NAME_EN', 'geometry']):
    
        # Get the largest polygon
        geometry = nu.get_largest_polygon(geometry)
    
        geometry_dict[country_name] = geometry
    
    # Initialize an empty graph
    adjacency_graph = nx.Graph()
    
    # Add nodes to the graph
    for country_name in country_names:
        adjacency_graph.add_node(country_name)
    
    # Get all unique pairs (order doesn't matter)
    country_pairs = combinations(country_names, 2)
    
    # For each pair
    for pair in country_pairs:
        country_name_from = pair[0]
        geometry_from = geometry_dict[country_name_from]
        country_name_to = pair[1]
        geometry_to = geometry_dict[country_name_to]
    
        # Check if they touch
        if geometry_from.touches(geometry_to):
    
            # Add an edge between the two countries
            adjacency_graph.add_edge(country_name_from, country_name_to)
    
    # Apply the greedy coloring algorithm
    coloring = nx.coloring.greedy_color(adjacency_graph, strategy='largest_first')
    
    # Get the node colors from the matplotlib prop cycle
    node_labels = [node for node in adjacency_graph.nodes()]
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    node_colors = [color_cycle[coloring[node] % len(color_cycle)] for node in adjacency_graph.nodes()]
    
    hex_color_dict = {}
    node_colors_dict = {}
    for node_label, hex_color in zip(node_labels, node_colors):
        IntegerRGB_obj = webcolors.hex_to_rgb(hex_color)
        node_color = (IntegerRGB_obj.red/255, IntegerRGB_obj.green/255, IntegerRGB_obj.blue/255)
        node_colors_dict[node_label] = node_color
    
    # Sort the nodes by highest degree first
    node_labels_colors = sorted(
        [(node_label, node_color) for node_label, node_color in node_colors_dict.items()],
        key=lambda x: adjacency_graph.degree(x[0]), reverse=True
    )
    
    # Remove the spread point closest to each node color in turn until you run out of spread points
    locations_list = [tuple(color) for color in spread_points[1:]]
    for country_name, node_color in node_labels_colors:
        nearest_neighbor = nu.get_nearest_neighbor(node_color, locations_list)
        rgb = locations_list.pop(locations_list.index(nearest_neighbor))
        rgb = (int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
        hex_color_dict[country_name] = webcolors.rgb_to_hex(rgb)
    
    # Create a separate hex color dictionary for use in the cartogram
    hex_color_dict['Ocean'] = '#87ceeb'
    
    return hex_color_dict

In [4]:

country_names = [
    'Bahrain', 'Egypt', 'Iran', 'Iraq', 'Israel', 'Jordan', 'Kuwait', 'Lebanon', 'Oman',
    'Qatar', 'Saudi Arabia', 'Syria', 'Turkey', 'United Arab Emirates', 'Yemen'
]
get_hex_color_dictionary(country_names, verbose=True)

100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


{'Saudi Arabia': '#0079b6', 'Iraq': '#ff7a00', 'Syria': '#009bc4', 'Israel': '#ff3900', 'Jordan': '#00c200', 'Oman': '#ff6785', 'Turkey': '#007c00', 'Iran': '#114b6d', 'Kuwait': '#00f377', 'Lebanon': '#002f00', 'United Arab Emirates': '#812b00', 'Yemen': '#00d5cf', 'Egypt': '#ffbfd5', 'Qatar': '#890000', 'Bahrain': '#e3e7ff', 'Ocean': '#87ceeb'}